In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

# core

> Fill in a module description here

# Load packages

Package requirements are stored in `requirements.yml`.

In [ ]:
from pathlib import Path

from lxml import etree

# Variables

BNC2014 needs to be downloaded for this script to work. It can be obtained from the official [BNC website](http://corpora.lancs.ac.uk/bnc2014/). 

The following variables need to be updated to the corpus' local path. In the current setting the BNC2014 data were stored in the project folder in the folder `data`, so relative paths were used.

In [ ]:
path_bnc = Path('../data/bnc-2014-spoken')
path_corpus = Path(path_bnc / 'spoken' / 'tagged')
path_metadata = Path(path_bnc / 'spoken' / 'metadata')

In [ ]:
assert path_bnc.exists()
assert path_corpus.exists()
assert path_metadata.exists()

# Load and parse XML

In [ ]:
path_texts = path_corpus.glob('*.xml')

In [ ]:
[next(path_texts) for _ in range(5)]

[PosixPath('../data/bnc-2014-spoken/spoken/tagged/SN64-tgd.xml'),
 PosixPath('../data/bnc-2014-spoken/spoken/tagged/SL8N-tgd.xml'),
 PosixPath('../data/bnc-2014-spoken/spoken/tagged/S59W-tgd.xml'),
 PosixPath('../data/bnc-2014-spoken/spoken/tagged/S79Y-tgd.xml'),
 PosixPath('../data/bnc-2014-spoken/spoken/tagged/S8K9-tgd.xml')]

In [ ]:
def get_xml(f_path):
    with open(f_path, 'r') as f:
        f = f.read()
    xml = etree.fromstring(f)
    return xml

In [ ]:
text = get_xml(next(path_texts))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()